# Transformers

Somewhat basic implemention of transformer model

In [ ]:
#| default_exp models.transformer

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
#| export

# torch
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.optim import SGD
from torchtext.vocab import vocab

# hf
import datasets

# data 
import pandas as pd
import numpy as np

# ui
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# python
from typing import Dict, List, Tuple, Optional, Set
from collections import Counter, OrderedDict
from dataclasses import dataclass, asdict
from plum import dispatch

# nimrod
from nimrod.models.lm import Vocab

## Data formatting
- https://buomsoo-kim.github.io/attention/2020/04/21/Attention-mechanism-19.md/


In [107]:
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Found cached dataset wikitext (/home/syl20/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': ''}


In [111]:
print(dataset['train'][88])

{'text': ' This ammunition , and that which I brought with me , was rapidly prepared for use at the Laboratory established at the Little Rock Arsenal for that purpose . As illustrating as the pitiful scarcity of material in the country , the fact may be stated that it was found necessary to use public documents of the State Library for cartridge paper . Gunsmiths were employed or conscripted , tools purchased or impressed , and the repair of the damaged guns I brought with me and about an equal number found at Little Rock commenced at once . But , after inspecting the work and observing the spirit of the men I decided that a garrison 500 strong could hold out against Fitch and that I would lead the remainder - about 1500 - to Gen \'l Rust as soon as shotguns and rifles could be obtained from Little Rock instead of pikes and lances , with which most of them were armed . Two days elapsed before the change could be effected . " \n'}


In [ ]:
# read unstructured text into pd
df = pd.read_fwf('../data/en/tiny_shakespeare.txt', header=None)
print("Dataframe: ", df.head(2))
data = df[0].tolist()
print("List: ", data[:2])
v = Vocab(data)
print("Vocab: ", len(v), ''.join(v.vocabulary))

Dataframe:                                                 0
0                                 First Citizen:
1  Before we proceed any further, hear me speak.
List:  ['First Citizen:', 'Before we proceed any further, hear me speak.']
Vocab:  68  !$&',-.3:;<bos><eos><pad><unk>?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:

# def make_dataset(names:List[str], verbose:bool=False, pad_value=0, context_length=3):
#     X = []
#     y = []
#     return torch.tensor(X),torch.tensor(y)
context_length = 3
print(data[0])
for i in range(len(data[0])-1):
    print(list(data[0])[i:context_length+i], list(data[0])[i+1:context_length+i+1])




First Citizen:
['F', 'i', 'r'] ['i', 'r', 's']
['i', 'r', 's'] ['r', 's', 't']
['r', 's', 't'] ['s', 't', ' ']
['s', 't', ' '] ['t', ' ', 'C']
['t', ' ', 'C'] [' ', 'C', 'i']
[' ', 'C', 'i'] ['C', 'i', 't']
['C', 'i', 't'] ['i', 't', 'i']
['i', 't', 'i'] ['t', 'i', 'z']
['t', 'i', 'z'] ['i', 'z', 'e']
['i', 'z', 'e'] ['z', 'e', 'n']
['z', 'e', 'n'] ['e', 'n', ':']
['e', 'n', ':'] ['n', ':']
['n', ':'] [':']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()